# T<sub>2</sub> Ramsey Experiment

This experiment serves as one of the series of experiments used to characterize a single qubit. Its purpose is to determine two of the qubit's properties: *Ramsey* or *detuning frequency* and $T_2\ast$. The rough frequency of the qubit was already determined previously. Here, we would like to measure the *detuning*, that is  the difference between the qubit's precise frequency and the frequency of the rotation pulses (based on the rough frequency). This part of the experiment is called a *Ramsey Experiment*. $T_2\ast$ represents the rate of decay toward a mixed state, when the qubit is initialized to the |+⟩ state.

In [1]:
import qiskit
from qiskit_experiments.characterization import T2Ramsey

The circuit used for the experiment comprises the following:

    1. Hadamard gate
    2. delay
    3. p (phase) gate that rotates the qubit in the x-y plane 
    4. Hadamard gate
    5. measurement

During the delay time, we expect the qubit to precess about the z-axis. If the p gate and the precession offset each other perfectly, then the qubit will arrive at the |0⟩ state (after the second Hadamard gate). By varying the extension of the delays, we get a series of oscillations of the qubit state between the |0⟩ and |1⟩ states. We can draw the graph of the resulting function, and can analytically extract the desired values.

In [2]:
# set the computation units to microseconds
unit = 'us' #microseconds
qubit = 0
# set the desired delays
delays = list(range(1, 150, 2))

In [3]:
# Create a T2Ramsey experiment. Print the first circuit as an example
exp1 = T2Ramsey(qubit, delays, unit=unit)
print(exp1.circuits()[0])

     ┌───┐┌──────────────┐┌──────┐ ░ ┌───┐ ░ ┌─┐
q_0: ┤ H ├┤ DELAY(1[us]) ├┤ P(0) ├─░─┤ H ├─░─┤M├
     └───┘└──────────────┘└──────┘ ░ └───┘ ░ └╥┘
c: 1/═════════════════════════════════════════╩═
                                              0 


We run the experiment on a simple, simulated backend, created specifically for this experiment's tutorial.

In [4]:
from qiskit_experiments.test.t2ramsey_backend import T2RamseyBackend
# FakeJob is a wrapper for the backend, to give it the form of a job
from qiskit_experiments.test.utils import FakeJob
import qiskit_experiments.matplotlib
from qiskit_experiments.matplotlib import pyplot, requires_matplotlib
from qiskit_experiments.matplotlib import HAS_MATPLOTLIB

conversion_factor = 1E-6
# The behavior of the backend is determined by the following parameters
backend = T2RamseyBackend(
                    p0={"a_guess":[0.5], "t2ramsey":[80.0], "f_guess":[0.02], "phi_guess":[0.0],
                        "b_guess": [0.5]},
                    initial_prob_plus=[0.0],
                    readout0to1=[0.02],
                    readout1to0=[0.02],
                    conversion_factor=conversion_factor,
                )


The resulting graph will have the form:
$ f(t) = a^{-t/T_2*} \cdot cos(2 \pi f t + \phi) + b $
where *t* is the delay, $T_2*$ is the decay factor, and *f* is the detuning frequency.
`conversion_factor` is a scaling factor that depends on the measurement units used. It is 1E-6 here, because the unit is microseconds.

In [5]:
exp1.set_analysis_options(user_p0=None, plot=True)
expdata1 = exp1.run(backend=backend, shots=2000)
expdata1.block_for_results()  # Wait for job/analysis to finish.
result = expdata1.analysis_results(0)
result_data = result.data()
# Print the result
result_data

{'t2ramsey_value': 8.119888734234995e-05,
 'frequency_value': 19986.285521503196,
 'stderr_t2': 1.1732134060237844e-06,
 'stderr_freq': 36.78671541710355,
 'unit': 's',
 'label': 'T2Ramsey',
 'fit': {'popt': array([ 4.80800910e-01,  8.11988873e-05,  1.99862855e+04, -3.54455132e-04,
          5.00308274e-01]),
  'popt_keys': None,
  'popt_err': array([3.08570554e-03, 1.17321341e-06, 3.67867154e+01, 1.10646030e-02,
         1.23281264e-03]),
  'pcov': array([[ 9.52157867e-06, -2.23211135e-09, -2.20537484e-02,
           9.47235941e-06, -6.05452911e-07],
         [-2.23211135e-09,  1.37642970e-12,  3.50045585e-06,
          -1.78851855e-09,  2.03470999e-10],
         [-2.20537484e-02,  3.50045585e-06,  1.35326243e+03,
          -3.13821712e-01, -1.16770978e-02],
         [ 9.47235941e-06, -1.78851855e-09, -3.13821712e-01,
           1.22425439e-04,  3.93775831e-06],
         [-6.05452911e-07,  2.03470999e-10, -1.16770978e-02,
           3.93775831e-06,  1.51982702e-06]]),
  'reduced_chisq

### Providing initial user estimates
The user can provide initial estimates for the parameters to help the analysis process. Because the curve is expected to decay toward $0.5$, the natural choice for parameters $A$ and $B$ is $0.5$. Varying the value of $\phi$ will shift the graph along the x-axis. Since this is not of interest to us, we can safely initialize $\phi$ to 0. In this experiment, `t2ramsey` and `f` are the parameters of interest. Good estimates for them are values computed in previous experiments on this qubit or a similar values computed for other qubits.

In [6]:
from qiskit_experiments.characterization import T2RamseyAnalysis
user_p0={
    "A": 0.5,
    "t2ramsey": 85.0,
    "f": 0.021,
    "phi": 0,
    "B": 0.5
        }
exp_with_p0 = T2Ramsey(qubit, delays, unit=unit)
exp_with_p0.set_analysis_options(user_p0=user_p0, plot=True)
expdata_with_p0 = exp_with_p0.run(backend=backend, shots=2000)
expdata_with_p0.block_for_results()
result = expdata_with_p0.analysis_results(0)
result_data = result.data()
# Print the result
result_data


{'t2ramsey_value': 7.915718737694317e-05,
 'frequency_value': 19987.370510637236,
 'stderr_t2': 1.1329893879207264e-06,
 'stderr_freq': 37.61732457909187,
 'unit': 's',
 'label': 'T2Ramsey',
 'fit': {'popt': array([4.81654953e-01, 7.91571874e-05, 1.99873705e+04, 2.73964461e-03,
         5.01027396e-01]),
  'popt_keys': None,
  'popt_err': array([3.04959810e-03, 1.13298939e-06, 3.76173246e+01, 1.11352926e-02,
         1.23582918e-03]),
  'pcov': array([[ 9.30004855e-06, -2.09098448e-09, -2.26320379e-02,
           9.62827838e-06, -6.65936823e-07],
         [-2.09098448e-09,  1.28366495e-12,  3.41842651e-06,
          -1.72684933e-09,  2.06353589e-10],
         [-2.26320379e-02,  3.41842651e-06,  1.41506311e+03,
          -3.22169480e-01, -1.18168766e-02],
         [ 9.62827838e-06, -1.72684933e-09, -3.22169480e-01,
           1.23994741e-04,  3.92756655e-06],
         [-6.65936823e-07,  2.06353589e-10, -1.18168766e-02,
           3.92756655e-06,  1.52727376e-06]]),
  'reduced_chisq': 1.

The units can be changed, but the output in the result is always given in seconds. The units in the backend must be adjusted accordingly.

In [7]:
from qiskit.utils import apply_prefix
unit = 'ns'
delays = list(range(1000, 150000, 2000))
conversion_factor = apply_prefix(1, unit)
print(conversion_factor)

1e-09


In [9]:
p0={"a_guess":[0.5], "t2ramsey":[80000], "f_guess":[0.00002], "phi_guess":[0.0],
                        "b_guess": [0.5]}
backend_in_ns = T2RamseyBackend(
                    p0=p0,
                    initial_prob_plus=[0.0],
                    readout0to1=[0.02],
                    readout1to0=[0.02],
                    conversion_factor=conversion_factor
                )
exp_in_ns = T2Ramsey(qubit, delays, unit=unit)
exp_in_ns.set_analysis_options(user_p0=None, plot=True)
expdata_in_ns = exp_in_ns.run(backend=backend_in_ns, shots=2000)
expdata_in_ns.block_for_results()
result = expdata_with_p0.analysis_results(0)
result_data = result.data()
result_data

{'t2ramsey_value': 7.915718737694317e-05,
 'frequency_value': 19987.370510637236,
 'stderr_t2': 1.1329893879207264e-06,
 'stderr_freq': 37.61732457909187,
 'unit': 's',
 'label': 'T2Ramsey',
 'fit': {'popt': array([4.81654953e-01, 7.91571874e-05, 1.99873705e+04, 2.73964461e-03,
         5.01027396e-01]),
  'popt_keys': None,
  'popt_err': array([3.04959810e-03, 1.13298939e-06, 3.76173246e+01, 1.11352926e-02,
         1.23582918e-03]),
  'pcov': array([[ 9.30004855e-06, -2.09098448e-09, -2.26320379e-02,
           9.62827838e-06, -6.65936823e-07],
         [-2.09098448e-09,  1.28366495e-12,  3.41842651e-06,
          -1.72684933e-09,  2.06353589e-10],
         [-2.26320379e-02,  3.41842651e-06,  1.41506311e+03,
          -3.22169480e-01, -1.18168766e-02],
         [ 9.62827838e-06, -1.72684933e-09, -3.22169480e-01,
           1.23994741e-04,  3.92756655e-06],
         [-6.65936823e-07,  2.06353589e-10, -1.18168766e-02,
           3.92756655e-06,  1.52727376e-06]]),
  'reduced_chisq': 1.

### Adding data to an existing experiment
It is possible to add data to an experiment, after the analysis of the first set of data. In the next example we add exp2 to `exp_in_ns` that we showed above.

In [10]:
more_delays = list(range(2000, 150000, 2000))         
exp_new = T2Ramsey(qubit, more_delays, unit=unit)
exp_new.set_analysis_options(user_p0=None, plot=True)
expdata_new = exp_new.run(
            backend=backend_in_ns,
            experiment_data=expdata_in_ns,
            shots=2000
        )
expdata_new.block_for_results()
result_new = expdata_new.analysis_results(1)
result_data = result_new.data()
result_data

{'t2ramsey_value': 8.006074592283946e-05,
 'frequency_value': 98980008.19039865,
 'stderr_t2': 8.411742088928867e-07,
 'stderr_freq': 26.417954346682556,
 'unit': 's',
 'label': 'T2Ramsey',
 'fit': {'popt': array([4.81691970e-01, 8.00607459e-05, 9.89800082e+07, 1.28565471e-03,
         5.00874085e-01]),
  'popt_keys': None,
  'popt_err': array([2.36119393e-03, 8.41174209e-07, 2.64179543e+01, 7.90057383e-03,
         8.78856370e-04]),
  'pcov': array([[ 5.57523678e-06, -1.27276267e-09,  1.34816502e-02,
          -5.69576563e-06, -2.46174537e-07],
         [-1.27276267e-09,  7.07574050e-13, -2.21112338e-06,
           1.07248306e-09,  8.92184069e-11],
         [ 1.34816502e-02, -2.21112338e-06,  6.97908312e+02,
          -1.61132471e-01,  6.09249965e-03],
         [-5.69576563e-06,  1.07248306e-09, -1.61132471e-01,
           6.24190668e-05, -2.05264748e-06],
         [-2.46174537e-07,  8.92184069e-11,  6.09249965e-03,
          -2.05264748e-06,  7.72388519e-07]]),
  'reduced_chisq': 1.0